# 项目开发规范

## 一些建议

1. 编码中考虑到其他python实现的效率等问题，比如运算符‘+’在CPython（Python）中效率很高，都是Jython中却非常低，所以应该采用.join()的方式。
2. 尽可能使用‘is’‘is not’取代‘==’，比如if x is not None 要优于if x。
3. 使用基于类的异常，每个模块或包都有自己的异常类，此异常类继承自Exception。
4. 异常中不要使用裸露的except，except后跟具体的exceptions。
5. 异常中try块中的代码尽可能少。
6. 字符串不要以空格收尾。

### 设计好的项目结构目录

```
Foo/
|-- bin/
|   |-- foo
|
|-- foo/
|   |-- tests/
|   |   |-- __init__.py
|   |   |-- test_main.py
|   |
|   |-- __init__.py
|   |-- main.py
|
|-- docs/
|   |-- conf.py
|   |-- abc.rst
|
|-- setup.py
|-- requirements.txt
|-- README
```

简要说明一下:

* bin/: 存放项目的一些可执行文件，当然你可以起名script/之类的也行。
* foo/: 存放项目的所有源代码。
    1. 源代码中的所有模块、包都应该放在此目录。不要置于顶层目录。
    2. 其子目录tests/存放单元测试代码； 
    3. 程序的入口最好命名为main.py。
* docs/: 存放一些文档。
* setup.py: 安装、部署、打包的脚本。
* requirements.txt: 存放软件依赖的外部Python包列表。
* README: 项目说明文件。

一个开源软件，目录该如何组织?
```
$ pwd
~/code/sandman
$ tree
.
|- LICENSE
|- README.md
|- TODO.md
|- docs
|   |-- conf.py
|   |-- generated
|   |-- index.rst
|   |-- installation.rst
|   |-- modules.rst
|   |-- quickstart.rst
|   |-- sandman.rst
|- requirements.txt
|- sandman
|   |-- __init__.py
|   |-- exception.py
|   |-- model.py
|   |-- sandman.py
|   |-- test
|       |-- models.py
|       |-- test_sandman.py
|- setup.py
```

### 关于README.md

几个事项：
1. 软件定位，软件的基本功能
2. 运行代码的方法：安装环境，启动命令等
3. 简要的使用说明
4. 代码目录结构说明，更详细点可以说明软件的基本原理
5. 常见问题说明

```
项目和所有子模块和库的名称（对于新用户，有时不同命名会导致混乱）
 
对所有项目，和所有子模块和库的描述
 
如何使用 5-line code（如果是一个库）
 
版权和许可信息（或阅读许可证）
 
抓取文档指令
 
安装、配置和运行程序的指导
 
抓取最新代码和构建它们的说明（或快速概述和「阅读 Install」）
 
作者列表或「Read AUTHORS」
 
提交bug，功能要求，提交补丁，加入邮件列表，得到通知，或加入用户或开发开发区群的介绍
 
其他联系信息（电子邮件地址，网站，公司名称，地址等）
 
一个简短的历史记录（更改，替换或者其他）
 
法律声明
```

### 关于requirements.txt和setup.py

#### setup.py

一般来说，用setup.py来管理代码的打包、安装、部署问题。

业界标准的写法是用Python流行的打包工具setuptools来管理这些事情。这种方式普遍应用于开源项目中。

不过这里的核心思想不是用标准化的工具来解决这些问题，而是说，一个项目一定要有一个安装部署工具，能快速便捷的在一台新机器上将环境装好、代码部署好和将程序运行起来。

setup.py可以将这些事情自动化起来，提高效率、减少出错的概率。"复杂的东西自动化，能自动化的东西一定要自动化。"是一个非常好的习惯。


可以参考一下Python的一个Web框架，flask是如何写的: [setup.py](https://github.com/pallets/flask/blob/master/setup.py)

当然，简单点自己写个安装脚本（deploy.sh）替代setup.py也未尝不可。

#### requirements.txt

这个文件存在的目的是:
1. 方便开发者维护软件的包依赖。将开发过程中新增的包添加进这个列表中，避免在 setup.py 安装依赖时漏掉软件包。
2. 方便读者明确项目使用了哪些Python包。

这个文件的格式是每一行包含一个包依赖的说明，通常是flask>=0.10这种格式，要求是这个格式能被pip识别，这样就可以简单的通过 pip install -r requirements.txt来把所有Python包依赖都装好了。

### 关于配置文件

注意，在上面的目录结构中，没有将conf.py放在源码目录下，而是放在docs/目录下。

很多项目对配置文件的使用做法是:

1. 配置文件写在一个或多个python文件中，比如此处的conf.py。
2. 项目中哪个模块用到这个配置文件就直接通过import conf这种形式来在代码中使用配置。

这种做法我不太赞同:

1. 这让单元测试变得困难（因为模块内部依赖了外部配置）
2. 另一方面配置文件作为用户控制程序的接口，应当可以由用户自由指定该文件的路径。
3. 程序组件可复用性太差，因为这种贯穿所有模块的代码硬编码方式，使得大部分模块都依赖conf.py这个文件。

所以，我认为配置的使用，更好的方式是，

1. 模块的配置都是可以灵活配置的，不受外部配置文件的影响。
2. 程序的配置也是可以灵活控制的。
能够佐证这个思想的是，用过nginx和mysql的同学都知道，nginx、mysql这些程序都可以自由的指定用户配置。

所以，
* 不应当在代码中直接import conf来使用配置文件。
* 上面目录结构中的conf.py，是给出的一个配置样例，不是在写死在程序中直接引用的配置文件。
* 可以通过给main.py启动参数指定配置路径的方式来让程序读取配置内容。当然，这里的conf.py你可以换个类似的名字，比如settings.py。或者你也可以使用其他格式的内容来编写配置文件，比如settings.yaml之类的。